# **Procesamiento Masivo de Datos con HDFS, Spark SQL y MLlib**

---

Este notebook debe quedar alojado en la carpeta GCS del cluster creado.

Selecciona el kernel de PySpark para ejecutarlo correctamente.

## Uso de Apache Spark

### Análisis y tratamiento de los datos

In [3]:
# El archivo flights.csv contiene información sobre vuelos, como el año, el mes, el día, la hora, el origen, el destino, la duración del vuelo, el retraso en la salida y el retraso en la llegada
ruta_hdfs = "/DataCluster_Test/flights.csv" # Ruta HDFS en la que se encuentra el archivo flights.csv

# Cargo el archivo flights.csv en un DataFrame con PySpark
# Indico que el archivo contiene encabezados y que intente inferir el esquema.
flightsDF = spark.read.option("header", "true").option("inferSchema", "true").csv(ruta_hdfs) 


Imprimo el esquema para comprobar si ha inferido correctamente el tipo de dato en cada columna.

In [4]:
flightsDF.printSchema() # Muestro el esquema

flightsDF.count() # Cuento la cantidad de registros

root
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- day: integer (nullable = true)
 |-- dep_time: string (nullable = true)
 |-- dep_delay: string (nullable = true)
 |-- arr_time: string (nullable = true)
 |-- arr_delay: string (nullable = true)
 |-- carrier: string (nullable = true)
 |-- tailnum: string (nullable = true)
 |-- flight: integer (nullable = true)
 |-- origin: string (nullable = true)
 |-- dest: string (nullable = true)
 |-- air_time: string (nullable = true)
 |-- distance: integer (nullable = true)
 |-- hour: string (nullable = true)
 |-- minute: string (nullable = true)



Veo que tenemos 162049 registros, coincide con el archivo. Si imprimo por pantalla las 5 primeras filas, veré qué tipos parecen tener y en qué columnas no coincide el tipo que podríamos esperar con el tipo que ha inferido Spark.

In [6]:
flightsDF.show(5)

+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
|2014|    1|  1|       1|       96|     235|       70|     AS| N508AS|   145|   PDX| ANC|     194|    1542|   0|     1|
|2014|    1|  1|       4|       -6|     738|      -23|     US| N195UW|  1830|   SEA| CLT|     252|    2279|   0|     4|
|2014|    1|  1|       8|       13|     548|       -4|     UA| N37422|  1609|   PDX| IAH|     201|    1825|   0|     8|
|2014|    1|  1|      28|       -2|     800|      -23|     US| N547UW|   466|   PDX| CLT|     251|    2282|   0|    28|
|2014|    1|  1|      34|       44|     325|       43|     AS| N762AS|   121|   SEA| ANC|     201|    1448|   0|    34|
+----+-----+---+--------+---------+-----

Al comparar lo inferido por Spark con el valor que tienen en el dataset algunas columnas, se puede ver que no les ha asignado el tipo de dato correctamente. La causa del problema es que en muchas columnas existe un valor faltante llamado "NA". Spark no reconoce ese valor como *no disponible* ni nada similar, sino que lo considera como un string de texto normal, y por tanto, asigna a toda la columna el tipo de dato string. 

Concretamente, las siguientes columnas deberían ser de tipo entero pero Spark las muestra como string:
<ul>
 <li>dep_time: string (nullable = true)
 <li>dep_delay: string (nullable = true)
 <li>arr_time: string (nullable = true)
 <li>arr_delay: string (nullable = true)
 <li>air_time: string (nullable = true)
 <li>hour: string (nullable = true)
 <li>minute: string (nullable = true)    
</ul>


Voy a averiguar cuántas filas tienen el valor "NA" (como string) en la columna dep_time por ejemplo:

In [4]:
from pyspark.sql import functions as F
cuantos_NA = flightsDF\
                .where(F.col("dep_time") == "NA")\
                .count()
cuantos_NA

857

Existen 857 filas que no tienen un dato válido en esa columna. Hay distintas maneras de trabajar con los valores faltantes, como por ejemplo imputarlos (reemplazarlos por un valor generado por nosotros según cierta lógica, por ejemplo la media de esa columna, etc). 

Lo más sencillo es quitar toda la fila, aunque esto depende de si nos lo podemos permitir en base a la cantidad de datos que tenemos. En este caso, el dataset dispone de un número considerable de filas en total en comparación con las que tienen valores nulos, así que quitaré todas las filas donde hay un NA en cualquiera de las columnas.

In [6]:
columnas_limpiar = ["dep_time", "dep_delay", "arr_time", "arr_delay", "air_time", "hour", "minute"]

flightsLimpiado = flightsDF
for nombreColumna in columnas_limpiar:  # Para cada columna, me quedo solo con las filas que no tienen NA en esa columna
    flightsLimpiado = flightsLimpiado.where(F.col(nombreColumna) != "NA")

flightsLimpiado.cache() # Guardo el DataFrame en memoria para que las operaciones sean más rápidas

24/05/09 10:51:23 WARN org.apache.spark.sql.execution.CacheManager: Asked to cache already cached data.


DataFrame[year: int, month: int, day: int, dep_time: string, dep_delay: string, arr_time: string, arr_delay: string, carrier: string, tailnum: string, flight: int, origin: string, dest: string, air_time: string, distance: int, hour: string, minute: string]

Una vez que se han eliminado los NA, voy a convertir a entero cada una las columnas que erróneamente eran de tipo string.

Ahora no debe haber problema ya que todas las cadenas de texto contienen dentro un número que puede ser convertido de texto a número. 
Convertiré también la columna `arr_delay` de entero a número real, será necesario para los pasos posteriores donde ajustaré un modelo predictivo.

In [8]:
from pyspark.sql.types import IntegerType, DoubleType

flightsConvertido = flightsLimpiado

for c in columnas_limpiar: # Método que crea una columna o reemplaza una existente con el mismo nombre, pero con un tipo de dato diferente
    flightsConvertido = flightsConvertido.withColumn(c, F.col(c).cast(IntegerType())) 

flightsConvertido = flightsConvertido.withColumn("arr_delay", F.col("arr_delay").cast(DoubleType())) # Cambio el tipo de dato de la columna arr_delay a DoubleType
flightsConvertido.cache()  # Cacheo el DataFrame para que las operaciones sean más rápidas

DataFrame[year: int, month: int, day: int, dep_time: int, dep_delay: int, arr_time: int, arr_delay: double, carrier: string, tailnum: string, flight: int, origin: string, dest: string, air_time: int, distance: int, hour: int, minute: int]

In [9]:
flightsConvertido.printSchema() # Muestro el esquema

root
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- day: integer (nullable = true)
 |-- dep_time: integer (nullable = true)
 |-- dep_delay: integer (nullable = true)
 |-- arr_time: integer (nullable = true)
 |-- arr_delay: double (nullable = true)
 |-- carrier: string (nullable = true)
 |-- tailnum: string (nullable = true)
 |-- flight: integer (nullable = true)
 |-- origin: string (nullable = true)
 |-- dest: string (nullable = true)
 |-- air_time: integer (nullable = true)
 |-- distance: integer (nullable = true)
 |-- hour: integer (nullable = true)
 |-- minute: integer (nullable = true)



Ahora el esquema si muestra el tipo de datos correcto en cada columna y Spark si está tratando como enteros las columnas que deberían serlo, y si quisiera podría hacer operaciones aritméticas
con ellas.

### Tareas varias

Partiendo del DataFrame `flightsConvertido` en cada tarea: 

#### Tarea 1

- Tarea 1 -> Crear un nuevo DataFrame llamado `aeropuertosOrigenDF` que tenga una columna `origin` y que tenga tantas filas como aeropuertos distintos de *origen* existan. ¿Cuántas filas tiene? Almacenar dicho recuento en la variable entera `n_origen`.

In [18]:
aeropuertosOrigenDF = flightsConvertido.select("origin").distinct()
n_origen = aeropuertosOrigenDF.count()
print(n_origen)

#### Tarea 2

- Tarea 2 -> Crear un nuevo DataFrame llamado `rutasDistintasDF` que tenga dos columnas `origin`, `dest` y que tenga tantas filas como rutas diferentes existan (es decir, como combinaciones distintas haya entre un origen y un destino). Una vez creado, contar cuántas combinaciones hay, almacenando dicho recuento en la variable entera `n_rutas`.

In [ ]:
rutasDistintasDF = flightsConvertido.select("origin", "dest").distinct()
n_rutas = rutasDistintasDF.count()
print(n_rutas)

#### Tarea 3

- Tarea 3 -> Calcular, *sólo para los vuelos que llegan con* ***retraso positivo***, el retraso medio a la llegada de dichos vuelos, para cada aeropuerto de destino. La nueva columna con el retraso medio a la llegada debe llamarse `retraso_medio`. El DF resultante debe estar ordenado de mayor a menor retraso medio. El código que calcule esto debería ir encapsulado en una función de Python llamada `retrasoMedio` que reciba como argumento un DataFrame y devuelva como resultado el DataFrame con el cálculo descrito anteriormente.
  
  Una vez hecha la función, invocarla pasándole como argumento `flightsConvertido` y almacenar el resultado devuelto en la variable `retrasoMedioDF`.

In [21]:
from pyspark.sql.functions import avg

def retrasoMedio(df):                          # Función que recibe un DataFrame y devuelve un nuevo DataFrame con el retraso medio por destino
    df_filtrado = df.filter(df.arr_delay > 0)
    df_resultado = df_filtrado.groupBy("dest") \
                              .agg(avg("arr_delay").alias("retraso_medio")) \
                              .orderBy("retraso_medio", ascending=False)
    return df_resultado

retrasoMedioDF = retrasoMedio(flightsConvertido) # DataFrame con el retraso medio por destino

Ahora llamo a la función `retrasoMedio` pasándole como argumento `flightsConvertido`. 
- ¿Cuáles son los tres aeropuertos con mayor retraso medio? 
- ¿Cuáles son sus retrasos medios en minutos?

In [23]:
retrasoMedioDF = retrasoMedio(flightsConvertido)
retrasoMedioDF.show(3)

+----+------------------+
|dest|     retraso_medio|
+----+------------------+
| BOI|             64.75|
| HDN|              46.8|
| SFO|41.193768844221104|
+----+------------------+
only showing top 3 rows



### Aplicación de Spark MLlib

A continuación voy a ajustar un modelo de DecisionTree de Spark para predecir si un vuelo vendrá o no con retraso (problema de clasificación binaria), utilizando como variables predictoras el mes, el día del mes, la hora de partida (`dep_time`), la hora de llegada (`arr_time`), el tipo de avión (`carrier`), la distancia y el tiempo que permanece en el aire.

#### Indexación

En estos datos hay variables numéricas y variables categóricas que ahora mismo están tipadas como numéricas, como por ejemplo el mes del año (`month`), que es en realidad categórica. Hay que indicar a Spark cuáles son categóricas e indexarlas. Para ello tengo que: 

- Crear un `StringIndexer` al que llamaré `indexerMonth` y otro al que llamaré `indexerCarrier` sobre las variables categóricas `month` y `carrier`. El nombre de las columnas indexadas que se van a crear son, respectivamente, `monthIndexed` y `carrierIndexed`.

In [27]:
from pyspark.ml.feature import StringIndexer # Importo la clase StringIndexer

indexerMonth = StringIndexer(inputCol="month", outputCol="monthIndexed") # Creo un objeto StringIndexer para la columna month

indexerCarrier = StringIndexer(inputCol="carrier", outputCol="carrierIndexed") # Creo un objeto StringIndexer para la columna carrier

#### Ensamble

Spark requiere que todas las variables estén en una única columna de tipo vector, por lo que después de indexar estas dos variables, hay que fusionarlas en una columna de tipo vector todas ellas, utilizando un `VectorAssembler`. Por tanto:

- Crearé en una variable llamada `vectorAssembler` un `VectorAssembler` que reciba como entrada una lista de todas las variables de entrada (sin incluir `arr_delay`) que serán las que formarán parte del modelo. 
  
    Crearé primero esta lista de variables (lista de strings) en la variable `columnas_ensamblar` y pasaré dicha variable como argumento al crear el `VectorAssembler`. En el caso de las columnas `month` y `carrier`, no usaré las variables originales sino las indexadas en el apartado anterior. La columna de tipo vector creada con las características ensambladas se llamará `features`.

In [28]:
from pyspark.ml.feature import VectorAssembler # Importo la clase VectorAssembler

columnas_ensamblar = ["monthIndexed", "carrierIndexed", "day", "dep_time", "arr_time", "distance", "air_time"] # Columnas que quiero ensamblar

vectorAssembler = VectorAssembler(inputCols=columnas_ensamblar, outputCol="features")  # Creo un objeto VectorAssembler

In [29]:
assert(isinstance(vectorAssembler, VectorAssembler))
assert(vectorAssembler.getOutputCol() == "features")
input_cols = vectorAssembler.getInputCols() 
assert(len(input_cols) == 7)
assert("arr_delay" not in input_cols)

#### Binarización de la columna objetivo

Finalmente, veo que la columna `arr_delay` es continua, y no binaria como requiere un problema de clasificación con dos clases. Por tanto, voy a convertirla en binaria. Para ello:

-  Utilizaré un binarizador de Spark, fijando a 15 el umbral, y guardarlo en la variable `delayBinarizer`. Consideraré retrasado un vuelo que llegue con más de 15 minutos de retraso, y no retrasado en caso contrario. La nueva columna creada con la variable binaria se llamará `arr_delay_binary` y será interpretada como la columna target para el algoritmo. Por ese motivo, esta columna **no** se incluyó en el apartado anterior entre las columnas que se ensamblan para formar las features.

In [30]:
from pyspark.ml.feature import Binarizer

delayBinarizer = Binarizer(threshold=15, inputCol="arr_delay", outputCol="arr_delay_binary") # Creo un objeto Binarizer para la columna arr_delay con un threshold de 15 minutos de retraso en la llegada

#### Creación del modelo de árbol de decisión

Por último, creo el modelo de clasificación.

- Creo en una variable `decisionTree` un árbol de clasificación de Spark (`DecisionTreeClassifier` del paquete `pyspark.ml.classification`)
- Indico como columna de entrada la columna creada por el `VectorAssembler`.
- Indico como columna objetivo (target) la columna creada por el `Binarizer`.

In [32]:
from pyspark.ml.classification import DecisionTreeClassifier

decisionTree = DecisionTreeClassifier(featuresCol="features", labelCol="arr_delay_binary") # Creo un objeto DecisionTreeClassifier con las columnas features y arr_delay_binary

Ahora encapsulo todas las fases en un sólo pipeline y lo entrenaré:

- Crearé en una variable llamada `pipeline` un objeto `Pipeline` de Spark con las etapas anteriores en el orden adecuado para poder entrenar un modelo. 

- Entrenarlo llamando al método `fit` y guardaré el pipeline entrenado devuelto en una variable llamada `pipelineModel`. 

- Aplico el pipeline entrenado para predecir el DataFrame `flightsConvertido`, guardando las predicciones devueltas en la variable `flightsPredictions` que será un DataFrame. 

- Crearé un evaluador con el fin de calcular la precisión del modelo.

In [34]:
from pyspark.ml import Pipeline

trainData, testData = flightsConvertido.randomSplit([0.8, 0.2], seed=7) # Divido los datos en entrenamiento y test (80% - 20%)

pipeline = Pipeline(stages=[indexerMonth, indexerCarrier, vectorAssembler, delayBinarizer, decisionTree]) # Creo un objeto Pipeline con los distintos objetos

pipelineModel = pipeline.fit(flightsConvertido) # Entreno el modelo

flightsPredictions = pipelineModel.transform(flightsConvertido) # Realizo las predicciones

flightsPredictions.select("features", "prediction", "arr_delay_binary").show() # Muestro algunas predicciones

Muestro la matriz de confusión. Agrupo por la variable que tiene la clase verdadera y la que tiene la clase predicha, para ver en cuántos casos coinciden y en cuántos difieren.

In [37]:
flightsPredictions.groupBy("arr_delay_binary", "prediction").count().show()

+----------------+----------+------+
|arr_delay_binary|prediction| count|
+----------------+----------+------+
|             1.0|       1.0|   530|
|             0.0|       1.0|    70|
|             1.0|       0.0| 23719|
|             0.0|       0.0|136429|
+----------------+----------+------+



In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator = MulticlassClassificationEvaluator( # Creo un evaluador para la precisión del modelo
    labelCol="arr_delay_binary",
    predictionCol="prediction",
    metricName="accuracy"
)

accuracy = evaluator.evaluate(flightsPredictions) # Calculo la precisión del modelo
print(f"Accuracy del modelo: {accuracy:.2f}")
